# The Social Web project

In this small project, we want to analyze the correlations between 'consipracy thinkers' on Twitter. We will look at certain hashtags that are likely correlated with consipracy theories in the Netherlands. In this notebook we will present all the steps that we have taken to find an answer to our research questions: 

1. Which words cause people to be more vulnerable to believe conspiracy theories?
2. How deeply clustered are conspiracy hashtags compared to regular hashtags?
3. What kind of hashtags are likely to be related to conspiracy theories?
4. How can we deal with misinformation on social media?

First we will install some packages to run all of our steps

In [32]:
import platform
import sys
print("This jupyter notebook is running on Python " + platform.python_version())
# It's good practice to assert packages requirements at the beginning of a script:
assert sys.version_info >= (3, 6) # Tested with Python==3.7.5

This jupyter notebook is running on Python 3.8.6


In [2]:
!pip3 install twitter PrettyTable matplotlib
!pip3 install requests
!pip3 install BeautifulSoup4
!pip3 install HTMLParser
!pip3 install rdflib
!pip3 install tweepy
!pip3 install sklearn

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
/Library/Frameworks/Python.framework/Versions/3.8/bin/python3: can't open file 'spacy': [Errno 2] No such file or directory


In [24]:
!pip3 install -U spacy
!python3 -m spacy download nl_core_news_sm

Requirement already up-to-date: spacy in /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages (2.3.4)
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
     |████████████████████████████████| 15.1 MB 10.4 MB/s eta 0:00:01   |████████▌                       | 4.0 MB 475 kB/s eta 0:00:24     |████████████████                | 7.5 MB 10.4 MB/s eta 0:00:01
Using legacy 'setup.py install' for nl-core-news-sm, since package 'wheel' is not installed.
    Running setup.py install for nl-core-news-sm ... done
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('nl_core_news_sm')


In [30]:
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
import string
import os
import time


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from string import punctuation
import collections
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nl_core_news_sm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Yossigijrath/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Yossigijrath/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


As we did in assignment 1 of the notebooks, we will import several hastags from Twitter. We firstly need to implement some Twitter API keys to make this possible.

In [35]:
# Twitter credentials
# Obtain them from your twitter developer account
consumer_key = 'N4J4jiQuGW7DZKLvUXg1DUYlw'
consumer_secret = 'HwhmSr8rGqbzG9NlfaOSHl4No2RijgtJwtDCQEFNZ5vlSPzYbj'
access_key = '158063657-YJjxwBnlFS6GGn0jkhnqA6PIjzVRC9UhACSjCWrX'
access_secret = 'kuwpm8QnoNIX3bTkgOrlhjbdEKYxRiFqxqRXXRdLGeoJF'
# Pass your twitter credentials to tweepy via its OAuthHandler
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

# 1. Lets take a look at some hashtags we see as consipracy believers

In our project we will look at the keyword: Viruswaarheid. This is a group that believes the Coronavirus is a hoax and that politicians are a group of pedophiles. 

Next to this we also want to find more information about the Young Forum for Democracy party which has been quite a lot in the media concerning antisemtism and consipricacy theories. 

We will first perform a scrape on Twitter and add the information in a CSV file. This file can be used in a later stage to analyze. 

In [36]:
tweets = tweepy.Cursor(api.search,q = 'Viruswaarheid', since_id = '2020-01-01').items(2000)
users = []
followers = []
friends = []
description = []
tweet_count = []
for status in tweets:
    name = status.user.screen_name
    desc = status.user.description
    number_of_tweets = status.user.statuses_count
    following = status.user.friends_count
    follower = status.user.followers_count
    users.append(name)
    followers.append(follower)
    friends.append(following)
    description.append(desc)
    tweet_count.append(number_of_tweets) 

data = {'screen_name':users, 'followers':followers, 'friends':friends, 'description':description, 'tweets':tweet_count}
df_8 = pd.DataFrame(data)
    
#     df['screen_name'] = users
#     df['location'] = location
    
df_8.to_csv('Viruswaarheid.csv')
df_8.head()

,screen_name,followers,friends,description,tweets
0,viruswaarheid,3269,109,@viruswaanzin gaat verder als @viruswaarheid\n...,2230
1,viruswaarheid,3269,109,@viruswaanzin gaat verder als @viruswaarheid\n...,2230
2,HumanBe00194645,1,2,"The end cannot justify the means, for the simp...",420
3,volger4444,23,136,,702
4,v4vendettaNL,32,62,,638


We will first create two files with tweets about #Viruswaarheid (conspiracy theory) and on the other hand a 'regular' hashtag named #Goedemorgen.

In [86]:
with open('Viruswaarheid-tweets.csv', 'w', newline='') as outcsv:
    writer = csv.writer(outcsv)
    writer.writerow(["ID", "Tweets"])
    for tweet in tweepy.Cursor(api.search,q="#Viruswaarheid",count=10000,
                           lang="nl",
                           since="2020-01-01").items():
        writer.writerow([tweet.id, tweet.text.encode('utf-8')])

In [87]:
with open('Goedemorgen-tweets.csv', 'w', newline='') as outcsv:
    writer = csv.writer(outcsv)
    writer.writerow(["ID", "Tweets"])
    for tweet in tweepy.Cursor(api.search,q="#Goedemorgen",count=10000,
                           lang="nl",
                           since="2020-01-01").items():
        writer.writerow([tweet.id, tweet.text.encode('utf-8')])

In [16]:
Viruswaarheid_tweets = pd.read_csv('Viruswaarheid-tweets.csv')
Goedemorgen_tweets = pd.read_csv('Goedemorgen-tweets.csv')

We will now perform some cleaning of the data, where we will use lemmatization, tokenizing, removing stopwards, punctuations, hashtags and mentions.

In [17]:
def clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df

clean_tweets_Viruswaarheid = clean_text(Viruswaarheid_tweets, 'Tweets')
clean_tweets_Viruswaarheid.head()

,ID,Tweets
0,1336408457101062151,bgoh sommige volgers van viruswaarheid stopwil...
1,1336384037661904897,bviruswaarheid wakeup riseup hoe houd rutte di...
2,1336363907112460292,b82 dansen met willem engel n via nnengel vir...
3,1336255744967110658,b okee dus op viruswaarheid nnja denk het ei...
4,1336054169493512192,brt b tijdrekken kan alleen als er gewacht w...


In [18]:
def clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df

clean_tweets_Goedemorgen = clean_text(Goedemorgen_tweets, 'Tweets')
clean_tweets_Goedemorgen.head()

,ID,Tweets
0,1336593342864486400,bgoede morgen tweepsff vraagje vinden jullie d...
1,1336589481063878656,bis het heel erg als nu al roep dat het bijna ...
2,1336588545117523970,bgoedemorgen deze woensdag nlekker hoor die it...
3,1336588395838124033,bgoedemorgen allemaal een mooie woensdag gewen...
4,1336587131817496577,bgoedemorgen mensen ik zie weinig vandaag mist...


In [32]:
nlp = nl_core_news_sm.load()
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words('dutch'))
punctuation = list(string.punctuation) #already taken care of with the cleaning function.
stop.update(punctuation)
w_tokenizer = WhitespaceTokenizer()

            
def furnished(text):
    final_text = []
    for i in w_tokenizer.tokenize(text):
#     for i in text.split():
        if i.lower() not in stop:
            word = lemmatizer.lemmatize(i)
            final_text.append(word.lower())
    return " ".join(final_text)


            
Viruswaarheid_tweets.Tweets = Viruswaarheid_tweets.Tweets.apply(furnished)

In [33]:
Viruswaarheid_tweets.sample(5)

,ID,Tweets
7,1335916147628236800,b ishere hart zegt nee verstand zegt denk mens...
2,1336363907112460292,b82 dansen willem engel n via nnengel viruswaa...
46,1333779652574277637,b gelezennik begrip aantal regels dxc3xadt gaa...
17,1335181390070079490,bvanaf minuut 34 nnpcrgate nviruswaarheidncoro...
31,1334515152973148163,bbeste nieuws vandaag nviruswaarheid viruswaan...


In [34]:
nlp = nl_core_news_sm.load()
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words('dutch'))
punctuation = list(string.punctuation) #already taken care of with the cleaning function.
stop.update(punctuation)
w_tokenizer = WhitespaceTokenizer()

            
def furnished(text):
    final_text = []
    for i in w_tokenizer.tokenize(text):
#     for i in text.split():
        if i.lower() not in stop:
            word = lemmatizer.lemmatize(i)
            final_text.append(word.lower())
    return " ".join(final_text)


            
Goedemorgen_tweets.Tweets = Goedemorgen_tweets.Tweets.apply(furnished)

In [35]:
Goedemorgen_tweets.sample(5)

,ID,Tweets
130,1336242730842910720,bkraken strafbaar volgens wet wet gewoon gehan...
1368,1333665547700015105,bzijn mensen geblokkeerd waarschijnlijk vonden...
1115,1334052646525743105,bgoedemorgenbijna sinterklaas afhankelijk tege...
772,1334775344864370689,bgoedemorgen allemaal iedereen fijne vrijdag w...
951,1334417963496202241,brt jouw geheim jong fit blijven ngoedemorgen ...


In [41]:
def lexical_diversity(tokens):
    return 1.0*len(set(tokens))/len(tokens)

# Define a function for computing the average number of words per tweet
def average_words(statuses):
    total_words = sum([ len(s.split()) for s in statuses ])
    return 1.0*total_words/len(statuses) 

# Let's use these functions:

print ('Lexical diversity #Goedemorgen tweets:', lexical_diversity(Goedemorgen_tweets.Tweets))
print ('Lexical diversity #Viruswaarheid tweets:', lexical_diversity(Viruswaarheid_tweets.Tweets))
print ('Average words #Goedemorgen tweets:', average_words(Goedemorgen_tweets.Tweets))
print ('Average words #Viruswaarheid tweets:', average_words(Viruswaarheid_tweets.Tweets))

Lexical diversity #Goedemorgen tweets: 0.81122763726095
Lexical diversity #Viruswaarheid tweets: 0.9649122807017544
Average words #Goedemorgen tweets: 9.794571252313387
Average words #Viruswaarheid tweets: 9.649122807017545
